In [1]:
from data import Ecoli_Kfold
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from keras.models import Sequential 
from keras.layers import Dense,Activation,Dropout 
from keras.utils import np_utils
from keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from datetime import datetime
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, f1_score, precision_score


In [3]:
dataset = Ecoli_Kfold
X, y = dataset.load_data()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
train_labels = to_categorical(y_train,num_classes=2)
test_labels = to_categorical(y_test,num_classes=2)
print("Shape of y_train",train_labels.shape)
print("Shape of y_test",test_labels.shape)

print(y_test)
print(test_labels)


Shape of y_train (235, 2)
Shape of y_test (101, 2)
[0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0.
 0. 0. 1. 0. 0.]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 

In [6]:
a = to_categorical([1,-1,1,1],num_classes=2)
print(a)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [4]:
model=Sequential()
model.add(Dense(1000,input_dim=7,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(300,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()
# model.fit(X_train,train_labels,validation_data=(X_test,test_labels),batch_size=20,epochs=10,verbose=1)
model.fit(X_train,train_labels,validation_data=(X_test,test_labels),batch_size=20,epochs=200,verbose=2)
date_stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = (str(dataset).split("\\")[-1]).split(".")[0]
# model.save(f'./Model/{filename}_{date_stamp}.h5') #lưu model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              8000      
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 dense_2 (Dense)             (None, 300)               150300    
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 602       
                                                                 
Total params: 659,402
Trainable params: 659,402
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
12/12 - 1s - loss: 0.5927 - accuracy: 

In [5]:
prediction=model.predict(X_test)
length=len(prediction)
y_label=np.argmax(test_labels,axis=1)
predict_label=np.argmax(prediction,axis=1)

accuracy=np.sum(y_label==predict_label)/length * 100 
print("Accuracy of the dataset",accuracy )

4/4 [==============================] - 0s 2ms/step
Accuracy of the dataset 91.0891089108911


In [6]:
import math
def compute_metrics(y_test,y_pred):
    cm_WSVM = confusion_matrix(y_test, y_pred)
    se = cm_WSVM[1,1]/(cm_WSVM[1,0]+cm_WSVM[1,1])
    sp = cm_WSVM[0,0]/(cm_WSVM[0,0]+cm_WSVM[0,1])
    gmean = math.sqrt(se*sp)
    f1s = f1_score(y_test,y_pred)
    acc = accuracy_score(y_test,y_pred)
    pre = precision_score(y_test,y_pred)
    auc = roc_auc_score(y_test, y_pred)

    return sp, se, gmean, f1s, acc, pre, auc, cm_WSVM

In [8]:

prediction = model.predict(X_test)
y_label=np.argmax(test_labels,axis=1)
print(y_label)

predict_label=np.argmax(prediction,axis=1)
print(predict_label)
print(confusion_matrix(y_label,predict_label))
sp, se, gmean, f1s, acc, pre, auc, cm_WSVM = compute_metrics(y_label,predict_label)
print("SP: ",sp)
print("SE: ",se)
print("G-means: ",gmean)
print("F1-Score: ",f1s)
print("Accuracy: ",acc)
print("AUC: ",auc)
print("Matrix Confusion: \n",cm_WSVM)


4/4 [==============================] - 0s 1ms/step
[0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0
 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0]
[[76  2]
 [ 7 16]]
SP:  0.9743589743589743
SE:  0.6956521739130435
G-means:  0.8232951710562281
F1-Score:  0.7804878048780488
Accuracy:  0.9108910891089109
AUC:  0.835005574136009
Matrix Confusion: 
 [[76  2]
 [ 7 16]]
